- load data into azureml (y)

    - get from there to azure ml
    - multiple ways?
    - 	 explain why blobstorage neccessary
    - (word on data duplication?)
    - set up azureml tldr:
        - make rsource group, workspace, launch  azML studio, + compute instance + notebook(on compute instance)
        - "Storage Blob Data Reader grants read access only. This allows loading data from files staged in the storage account.
Storage Blob Data Contributor"
    - exasol -> blobstorage -> azureML
    - 
    
azurml:
    - create
    - compute (link to tut)
    - notebook (link to tut)
    - in compute find public ip
    add to exasol saas like local above
